In [1]:
import pandas as pd
import matplotlib as pyplot   
from pathlib import Path
import requests
import json
from census import Census
from us import states

In [2]:
api_key = "1a075ccdd14f7502695aa6696d73d28a29c0565e"
# base_url = "http://api.census.gov/data.json"
base_url = "http://api.census.gov/data/2022/cps/asec/mar"
response = requests.get(base_url)
print(response.url)

https://api.census.gov/data/2022/cps/asec/mar


In [3]:
census_client = Census(api_key)
state_code = states.CA.fips

variables = [
    "NAME",
    "B15003_002E", # Number of people with no high school diploma
    "B15003_022E",  # Number of people with a Bachelor's degree
    "B15003_025E",  # Number of people with a graduate degree
    # Add more variables as needed
]

# Fetch the data for all counties in California
data = census_client.acs5.state_county(variables, state_code, Census.ALL, year = 2021)

# Create a DataFrame from the fetched data
df = pd.DataFrame(data)

# Print the DataFrame
print(df.head())

                           NAME  B15003_002E  B15003_022E  B15003_025E state  \
0    Alameda County, California      31416.0     337847.0      36967.0    06   
1     Alpine County, California         11.0        200.0         41.0    06   
2     Amador County, California        376.0       4317.0        173.0    06   
3      Butte County, California       2195.0      27819.0       1813.0    06   
4  Calaveras County, California        250.0       4876.0        232.0    06   

  county  
0    001  
1    003  
2    005  
3    007  
4    009  


In [4]:
len(df)

58

In [5]:
import urllib.request
url = 'https://data.ca.gov/api/3/action/datastore_search?resource_id=57142555-f2da-462f-a999-d44abf0af69c&limit=5&q=title:jones'  
fileobj = urllib.request.urlopen(url)
response_dict = json.loads(fileobj.read()) # Creates python object
print(json.dumps(response_dict, indent=4, sort_keys=True))

{
    "help": "https://data.ca.gov/api/3/action/help_show?name=datastore_search",
    "result": {
        "_links": {
            "next": "/api/3/action/datastore_search?q=title%3Ajones&offset=5&limit=5&resource_id=57142555-f2da-462f-a999-d44abf0af69c",
            "start": "/api/3/action/datastore_search?q=title%3Ajones&limit=5&resource_id=57142555-f2da-462f-a999-d44abf0af69c"
        },
        "fields": [
            {
                "id": "_id",
                "type": "int"
            },
            {
                "id": "CALENDAR_YEAR",
                "info": {
                    "label": "Calendar Year",
                    "notes": "Valid Values:2017, 2018, 2019, 2020, 2021, 2022",
                    "type_override": "numeric"
                },
                "type": "numeric"
            },
            {
                "id": "COC_ID",
                "info": {
                    "label": "Continuum of Care Number",
                    "notes": "Valid Values: Any Cal

In [7]:
# THIS GIVES AN EMPTY DATA FRAME.....
data = response_dict

records = data['result']['records']

# Create an empty list to store the extracted information
extracted_data = []

# Loop through the records and extract the desired information
for record in records:
    if record['CALENDAR_YEAR'] == 2021:
        coc_name = record['COC_NAME']
        homeless_count = record['EXPERIENCING_HOMELESSNESS_CNT']
        extracted_data.append({'COC Name': coc_name, 'Homeless Count': homeless_count})
print(extracted_data)
# # Create a DataFrame from the extracted data
# df = pd.DataFrame(extracted_data)

# # Print the DataFrame
# print(df)

[]


In [ ]:
# file_path = "C:/Users/evacs/Downloads/2007-2022-PIT-Counts-by-CoC.xlsx"
# county_data = pd.read_excel(file_path, sheet_name = "2022")
# county_data.head()

In [ ]:
# ca_county_data = county_data[county_data['CoC Number'].str.contains('CA', na=False)]

# print(len(ca_county_data))
# ca_county_data.head()


In [ ]:
# file_path = "C:/Users/evacs/Downloads/2007-2022-PIT-Counts-by-CoC.xlsx"
# county_data = pd.read_excel(file_path, sheet_name = "2022")
# county_data.head()

In [ ]:
# import json
# import urllib.request
# url = 'https://data.chhs.ca.gov/api/3/action/datastore_search?resource_id=ad8277be-9e97-4904-9954-006ba8e35dd8&limit=5&q=title:jones'  
# fileobj = urllib.request.urlopen(url)
# response_dict = json.loads(fileobj.read())
# print(response_dict)

In [ ]:
#print(json.dumps(response_dict, indent=4, sort_keys=True))